<a href="https://colab.research.google.com/github/nitinware/GoogleCloabNotebooks/blob/master/pytorch_create_save_onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv('https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/storepurchasedata_large.csv')

In [3]:
dataset.describe()

,Age,Salary,Purchased
count,1554.000000,1554.000000,1554.000000
mean,44.296010,57042.471042,0.694981
std,17.462458,21209.244800,0.460564
min,18.000000,20000.000000,0.000000
25%,27.000000,46000.000000,0.000000
50%,43.000000,60000.000000,1.000000
75%,62.000000,66000.000000,1.000000
max,69.000000,96000.000000,1.000000


In [4]:
dataset.head()

,Age,Salary,Purchased
0,18,20000,0
1,19,22000,0
2,20,24000,0
3,21,28000,0
4,22,60000,1


In [5]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:,-1].values

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =.20,random_state=0)

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [8]:
Xtrain_ = torch.from_numpy(X_train).float()
Xtest_ = torch.from_numpy(X_test).float()

In [9]:
Xtrain_

tensor([[-0.0935,  0.2215],
        [ 1.2217, -0.5342],
        [-1.0655,  0.4104],
        ...,
        [ 1.2789,  0.4104],
        [-0.9512,  0.2215],
        [-1.2943, -1.4316]])

In [10]:
ytrain_ = torch.from_numpy(y_train)
ytest_ = torch.from_numpy(y_test)

In [11]:
ytrain_

tensor([1, 1, 1,  ..., 1, 1, 0])

In [12]:
Xtrain_.shape, ytrain_.shape

(torch.Size([1243, 2]), torch.Size([1243]))

In [13]:
Xtest_.shape, ytest_.shape

(torch.Size([311, 2]), torch.Size([311]))

In [14]:
input_size=2
output_size=2
hidden_size=10

In [15]:
class Net(nn.Module):
   def __init__(self):
       super(Net, self).__init__()
       self.fc1 = torch.nn.Linear(input_size, hidden_size)
       self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
       self.fc3 = torch.nn.Linear(hidden_size, output_size)


   def forward(self, X):
       X = torch.relu((self.fc1(X)))
       X = torch.relu((self.fc2(X)))
       X = self.fc3(X)

       return F.log_softmax(X,dim=1)

In [16]:
model = Net()



In [17]:
import torch.optim as optim
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.NLLLoss()

In [18]:
epochs = 100


In [19]:
for epoch in range(epochs):
  optimizer.zero_grad()
  Ypred = model(Xtrain_)
  loss = loss_fn(Ypred,  ytrain_)
  loss.backward()
  optimizer.step()
  print('Epoch',epoch, 'loss',loss.item())

Epoch 0 loss 0.6575396656990051
Epoch 1 loss 0.636155366897583
Epoch 2 loss 0.6184004545211792
Epoch 3 loss 0.6041350364685059
Epoch 4 loss 0.5923020243644714
Epoch 5 loss 0.5812477469444275
Epoch 6 loss 0.5711288452148438
Epoch 7 loss 0.5616785883903503
Epoch 8 loss 0.5524254441261292
Epoch 9 loss 0.5434041023254395
Epoch 10 loss 0.5341464877128601
Epoch 11 loss 0.524608314037323
Epoch 12 loss 0.5145539045333862
Epoch 13 loss 0.5038907527923584
Epoch 14 loss 0.492177814245224
Epoch 15 loss 0.4793303310871124
Epoch 16 loss 0.4654332399368286
Epoch 17 loss 0.4501419961452484
Epoch 18 loss 0.4334118366241455
Epoch 19 loss 0.41573992371559143
Epoch 20 loss 0.3974701762199402
Epoch 21 loss 0.3788011074066162
Epoch 22 loss 0.35973265767097473
Epoch 23 loss 0.34072327613830566
Epoch 24 loss 0.3218757212162018
Epoch 25 loss 0.30341836810112
Epoch 26 loss 0.28589195013046265
Epoch 27 loss 0.26975518465042114
Epoch 28 loss 0.25527697801589966
Epoch 29 loss 0.2424742579460144
Epoch 30 loss 0.231

In [20]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.0731,  0.4321],
         [-0.0852,  1.0153],
         [-0.5381, -0.5728],
         [ 0.2923,  0.4873],
         [ 1.3449,  0.0662],
         [-0.8132, -0.0037],
         [-0.3046,  0.3611],
         [-0.3843, -0.4157],
         [-0.8651, -1.1066],
         [ 0.3106,  0.6853]], requires_grad=True), Parameter containing:
 tensor([ 1.0083,  0.5496,  0.1792, -0.3419,  0.2387, -0.1518, -0.7953,  0.4682,
          0.0542, -0.1970], requires_grad=True), Parameter containing:
 tensor([[ 0.1299, -0.0650,  0.2119,  0.0531,  0.0105, -0.0798,  0.2442, -0.1595,
          -0.3228, -0.2786],
         [ 0.0457, -0.0071, -0.1057, -0.0828, -0.3114,  0.1642, -0.1870, -0.2399,
          -0.0737,  0.1578],
         [-0.2801, -0.0956, -0.1454, -0.0928, -0.2307, -0.1164,  0.0393,  0.1157,
          -0.0168, -0.0515],
         [-0.3630,  0.1228,  0.0826,  0.3110,  1.0764,  0.3174,  0.2309,  0.1706,
           0.4249,  0.5971],
         [ 0.5042,  0.7868, -0.4015,  0.0979,  

In [21]:
torch.from_numpy(sc.transform(np.array([[40,20000]]))).float()

tensor([[-0.2650, -1.7622]])

In [22]:
y_cust_20_40000 = model(torch.from_numpy(sc.transform(np.array([[40,20000]]))).float())
y_cust_20_40000

tensor([[-0.0102, -4.5908]], grad_fn=<LogSoftmaxBackward>)

In [24]:
_, predicted_20_40000 = torch.max(y_cust_20_40000.data,-1)
predicted_20_40000

tensor([0])

In [25]:
y_cust_42_50000 = model(torch.from_numpy(sc.transform(np.array([[42,50000]]))).float())
y_cust_42_50000

tensor([[-0.7168, -0.6700]], grad_fn=<LogSoftmaxBackward>)

In [26]:
_, predicted_42_50000 = torch.max(y_cust_42_50000.data,-1)
predicted_42_50000

tensor([1])

Export model to ONNX format

In [27]:
sample_tensor = torch.from_numpy(sc.transform(np.array([[40,20000]]))).float()


In [28]:
torch.onnx.export(model,sample_tensor,"customer.onnx",export_params=True)


In [29]:
!ls

customer.onnx  sample_data


Load the model to TensorFlow

In [30]:
!pip install onnx


     |████████████████████████████████| 12.2MB 11.9MB/s 


In [31]:
!pip install onnx_tf


     |████████████████████████████████| 225kB 11.8MB/s 
     |████████████████████████████████| 686kB 22.1MB/s 


In [32]:
import onnx
import onnx_tf


In [33]:
import tensorflow_addons

In [34]:
from onnx_tf.backend import prepare


In [35]:
onnx_model = onnx.load("customer.onnx")


In [36]:
tf_rep = prepare(onnx_model)


In [37]:
prediction = tf_rep.run((sc.transform(np.array([[42,50000]]))))


RuntimeError: ignored

In [38]:
prediction

NameError: ignored